# Installing required packages

In [ ]:
!pip install nemo_toolkit[all]
!pip install pydub
!pip install jsonlines
!pip install jiwer
!pip install pyaspeller
!wget https://sc.link/ZMv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 101.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.1/151.1 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2023-05-14 09:25:05--  https://sc.link/ZMv
Resolving sc.link (sc.link)... 37.230.233.245
Connecting to sc.link (sc.link)|37.230.233.245|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://n-ws-3jtx8.s3pd02.sbercloud.ru/b-ws-3jtx8-eir/golos/QuartzNet15x5_golos.nemo [following]
--2023-05-14 09:25:06--  https://n-ws-3jtx8.s3pd02.sbercloud.ru/b-ws-3jtx8-eir/golos/QuartzNet15x5_golos.nemo
Resolving n-ws-3jtx8.s3pd02.sbercloud.ru (n-ws-3jtx8.s3pd02.sbercloud.ru)... 37.18.122.129
Connecting to n-ws-3jtx8.s3pd02.sbercloud.ru (n-ws-3jtx8.s3pd02.sbercloud.ru)|37.18.122.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71179174 (68M) [application/octet-stream]
Saving to: ‘ZMv’

ZMv                 100%[===================>]  67.88M  16.7MB/s    in 4.1s    

2023-05-14 09:25:12 (16.7 MB/s) - ‘ZMv’ saved [71179174/71179174]



In [ ]:
import re
import os
import librosa
import nemo
import nemo.collections.asr as nemo_asr
import jsonlines
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from jiwer import wer, cer
import pytorch_lightning as pl
from omegaconf import DictConfig
from pyaspeller import YandexSpeller

[NeMo W 2023-05-14 09:25:38 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-05-14 09:25:41 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.


# Define the model

In [ ]:
asr_model = nemo_asr.models.EncDecCTCModel.restore_from("/content/ZMv")

[NeMo W 2023-05-14 09:25:46 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: train/golos_and_mcv.jsonl
    sample_rate: 16000
    labels:
    - ' '
    - а
    - б
    - в
    - г
    - д
    - е
    - ж
    - з
    - и
    - й
    - к
    - л
    - м
    - н
    - о
    - п
    - р
    - с
    - т
    - у
    - ф
    - х
    - ц
    - ч
    - ш
    - щ
    - ъ
    - ы
    - ь
    - э
    - ю
    - я
    batch_size: 64
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    num_workers: 20
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    parser: ru
    
[NeMo W 2023-05-14 09:25:46 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a

[NeMo I 2023-05-14 09:25:46 features:287] PADDING: 16
[NeMo I 2023-05-14 09:25:48 save_restore_connector:249] Model EncDecCTCModel was successfully restored from /content/ZMv.


# Dowload the manifest

Manifest contains path to audio, transcription and duration of the audio.

In [ ]:
manifest = []
with jsonlines.open('/content/drive/MyDrive/manifest.jsonl') as f:
    for obj in f:
        manifest.append(obj)

# Test the model without fine-tuning

In [ ]:
files = [file['audio_filepath'] for file in manifest]
transcriptions = asr_model.transcribe(paths2audio_files=files)

Transcribing:   0%|          | 0/1981 [00:00<?, ?it/s]

In [ ]:
wers = []
cers = []

for i, transcription in enumerate(transcriptions):
    if manifest[i]['text'] != '' and manifest[i]['text'] != ' ':
      w = wer(manifest[i]['text'], transcription)
      wers.append(w)
      c = cer(manifest[i]['text'], transcription)
      cers.append(c)
      manifest[i]['transcript'] = transcription

print('Mean WER: ', sum(wers)/len(wers))
print('Mean CER: ', sum(cers)/len(cers))

Mean WER:  0.8197088137628972
Mean CER:  0.5883378402583683


# Train

## Define trainer from pytorch lightning

In [ ]:
trainer = pl.Trainer(max_epochs=20, accelerator="cuda")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


## Define model

In [ ]:
asr_model = nemo_asr.models.EncDecCTCModel.restore_from("/content/ZMv")

[NeMo W 2023-05-14 09:26:35 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: train/golos_and_mcv.jsonl
    sample_rate: 16000
    labels:
    - ' '
    - а
    - б
    - в
    - г
    - д
    - е
    - ж
    - з
    - и
    - й
    - к
    - л
    - м
    - н
    - о
    - п
    - р
    - с
    - т
    - у
    - ф
    - х
    - ц
    - ч
    - ш
    - щ
    - ъ
    - ы
    - ь
    - э
    - ю
    - я
    batch_size: 64
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    num_workers: 20
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    parser: ru
    
[NeMo W 2023-05-14 09:26:35 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a

[NeMo I 2023-05-14 09:26:35 features:287] PADDING: 16
[NeMo I 2023-05-14 09:26:36 save_restore_connector:249] Model EncDecCTCModel was successfully restored from /content/ZMv.


## Split data to train and test samples

In [ ]:
train, test = train_test_split(manifest, test_size=0.3, random_state=22)

with jsonlines.open('train.jsonl', 'w') as writer:
    writer.write_all(train)
with jsonlines.open('test.jsonl', 'w') as writer:
    writer.write_all(test)

## Change parameters in the config

In [ ]:
max_dur = 0
for m in manifest:
    if m['duration'] > max_dur:
        max_dur = m['duration']

params = asr_model._cfg
params['optim']['lr'] = 0.001
params['spec_augment']['rect_masks'] = 0
params['train_ds']['parser']='ru'
params['train_ds']['manifest_filepath'] = '/content/train.jsonl'
params['train_ds']['batch_size']=8
params['validation_ds']['parser']='ru'
params['validation_ds']['manifest_filepath'] = '/content/test.jsonl'
params['validation_ds']['batch_size']=8
params['validation_ds']['num_workers'] = 0
params['test_ds']['parser']='ru'
params['test_ds']['manifest_filepath'] = '/content/test.jsonl'
params['test_ds']['batch_size']=8
params['train_ds']['num_workers'] = 0
params['train_ds']['max_duration'] = max_dur
params['train_ds']['min_duration'] = 0.01

In [ ]:
cfg_ = DictConfig(params)

## Restore model with our new config

In [ ]:
asr_model = nemo_asr.models.EncDecCTCModel.restore_from('/content/ZMv', override_config_path=cfg_)
asr_model.setup_training_data(cfg_['train_ds'])
asr_model.setup_validation_data(cfg_['validation_ds'])
asr_model.setup_test_data(cfg_['test_ds'])

[NeMo W 2023-05-14 09:26:59 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /content/train.jsonl
    sample_rate: 16000
    labels:
    - ' '
    - а
    - б
    - в
    - г
    - д
    - е
    - ж
    - з
    - и
    - й
    - к
    - л
    - м
    - н
    - о
    - п
    - р
    - с
    - т
    - у
    - ф
    - х
    - ц
    - ч
    - ш
    - щ
    - ъ
    - ы
    - ь
    - э
    - ю
    - я
    batch_size: 8
    trim_silence: false
    max_duration: 37.02
    min_duration: 0.01
    num_workers: 0
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    parser: ru
    
[NeMo W 2023-05-14 09:26:59 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a vali

[NeMo I 2023-05-14 09:26:59 features:287] PADDING: 16
[NeMo I 2023-05-14 09:27:01 save_restore_connector:249] Model EncDecCTCModel was successfully restored from /content/ZMv.
[NeMo I 2023-05-14 09:27:01 collections:193] Dataset loaded with 5545 files totalling 4.65 hours
[NeMo I 2023-05-14 09:27:01 collections:194] 0 files were filtered totalling 0.00 hours
[NeMo I 2023-05-14 09:27:02 collections:193] Dataset loaded with 2377 files totalling 2.02 hours
[NeMo I 2023-05-14 09:27:02 collections:194] 0 files were filtered totalling 0.00 hours
[NeMo I 2023-05-14 09:27:02 collections:193] Dataset loaded with 2377 files totalling 2.02 hours
[NeMo I 2023-05-14 09:27:02 collections:194] 0 files were filtered totalling 0.00 hours


## Train model

In [ ]:
trainer.fit(asr_model)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2023-05-14 09:27:24 modelPT:722] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.001
        weight_decay: 0.001
    )
[NeMo I 2023-05-14 09:27:24 lr_scheduler:910] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7fb2add2b430>" 
    will be used during training (effective maximum steps = 13880) - 
    Parameters : 
    (warmup_steps: 500
    warmup_ratio: null
    min_lr: 0.0
    last_epoch: -1
    max_steps: 13880
    )


INFO:pytorch_lightning.callbacks.model_summary:
  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18.9 M
2 | decoder           | ConvASRDecoder                    | 34.9 K
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WER                               | 0     
------------------------------------------------------------------------
18.9 M    Trainable params
0         Non-trainable params
18.9 M    Total params
75.718    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


## Save model to disk

In [ ]:
!cp /content/lightning_logs/version_0/checkpoints/epoch=19-step=13880.ckpt /content/drive/MyDrive

# Evaluate

In [ ]:
files = [file['audio_filepath'] for file in test]
transcriptions = asr_model.transcribe(paths2audio_files=files)

Transcribing:   0%|          | 0/595 [00:00<?, ?it/s]

In [ ]:
wers = []
cers = []

for i, transcription in enumerate(transcriptions):
    if test[i]['text'] != '' and test[i]['text'] != ' ':
      w = wer(test[i]['text'], transcription)
      wers.append(w)
      c = cer(test[i]['text'], transcription)
      cers.append(c)
      test[i]['transcript'] = transcription

print('Mean WER: ', sum(wers)/len(wers))
print('Mean CER: ', sum(cers)/len(cers))

Mean WER:  0.5110837006460176
Mean CER:  0.25160056654058255


# Use spellchecker for the received transcriptions

In [ ]:
speller = YandexSpeller()
transcrtiptions_spelled = []
for t in tqdm(transcriptions):
    transcrtiptions_spelled.append(speller.spelled(t))

100%|██████████| 2377/2377 [28:16<00:00,  1.40it/s]


In [ ]:
wers = []
cers = []

for i, transcrtiption_spelled in enumerate(transcrtiptions_spelled):
    if test[i]['text'] != '' and test[i]['text'] != ' ':
      w = wer(test[i]['text'], transcrtiption_spelled)
      wers.append(w)
      c = cer(test[i]['text'], transcrtiption_spelled)
      cers.append(c)
      test[i]['transcript'] = transcrtiption_spelled

print('Mean WER: ', sum(wers)/len(wers))
print('Mean CER: ', sum(cers)/len(cers))

Mean WER:  0.46444370782003275
Mean CER:  0.2532467376531301
